# Objectives
1. To observe how the price of various produce has changed overtime. To do this, I am going to collect data from [producepriceindex.com](https://www.producepriceindex.com) which cites its sources for this data as the USDA Market News Reports for Farm Gate Prices & U.S. Marketing Services for Retail Prices. The data appears to cover weekly prices from January 2000 to around May 2019. I don't see an option to download this data and am going to collect it via BeautifulSoup.

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd # Need to export data as df

In [2]:
URL = "http://www.producepriceindex.com/"

In [3]:
uClient = urlopen(URL) # Opens first page
html = uClient.read() # Reads in all HTML for that page
uClient.close() # Close connection
page_soup = soup(html, 'html.parser') # Create soup object
data = page_soup.find_all('td') # Grab the table data section
data # Shows the first page of website

[<td class="fix-width-16">
             Strawberries          </td>, <td class="fix-width-12">
 <span class="date-display-single" content="2019-05-19T00:00:00-07:00" datatype="xsd:dateTime" property="dc:date">2019-05-19</span> </td>, <td class="fix-width-12">
             $1.16          </td>, <td class="fix-width-12">
             $2.23          </td>, <td class="fix-width-12">
             $1.70          </td>, <td class="fix-width-12">
             $1.99          </td>, <td class="fix-width-12">
             $2.54          </td>, <td class="views-field views-field-expression-4">
             82.33%          </td>, <td class="fix-width-16">
             Romaine Lettuce          </td>, <td class="fix-width-12">
 <span class="date-display-single" content="2019-05-19T00:00:00-07:00" datatype="xsd:dateTime" property="dc:date">2019-05-19</span> </td>, <td class="fix-width-12">
             $0.35          </td>, <td class="fix-width-12">
             $1.72          </td>, <td class="fix-wi

In [4]:
# Grab all page urls. There are 637 pages
urls = ['http://www.producepriceindex.com/']
for i in range(1, 638):
    urls.append(URL+f'produce-price-index?field_ppi_commodity_target_id=All&field_ppi_date_value%5Bmin%5D=&field_ppi_date_value%5Bmax%5D=&page={i}')

Create a dictionary to store commodity data. Each produce item will serve as a key in this dictionary
The value data will be a list of the date followed by the farm price, Atlanta, Chicago, LA, and NYC retail prices,
and finally the average spread. This list will repeat these 7 data points each time a commodity shows up on a page
For example, for strawberries, the entry will look something like

```{"strawberries": ['2019-05-01', '$0.89', '$2.09', '$2.49', '$3.00', '$3.01', '139.82%', "2019-04-20", ...]}```

Will have to fix the data types and configure it into a dataframe later. Collect data into the dictionary "produce" for now

In [5]:
produce = dict() # create an empty dictionary to hold all data

# For each page on the website, read in the soup object and find all table data
for page in urls:
    uClient = urlopen(page)
    html = uClient.read()
    uClient.close()
    page_soup = soup(html, 'html.parser')
    data = page_soup.find_all('td')
    
    # When you have the html table data, loop through all 25 rows
    for j in range(0, 25):
        commodity = []
        for i in data[0 + 8*j].contents[0].split(' '): # Pick out the name of the commodity (happens every 8 iterations)
            if (i != '') & (i != '\n'):
                commodity.append(i) # Pick out only the name, ditch the white spaces and newline


        # If the commodity name is one word long, set the dictionary with that word. If not already set, create an empty list
        if len(commodity) == 1: 
            word = commodity[0]
            if word not in produce:
                produce[word] = []
            produce[word].append(data[1 + 8*j].span.string) # append the date to the list. Found at position 1 and every 8 iterations thereafter

        # otherwise if the commodity is 2 words, fix this into a single item to serve as key. Then do the same as above
        elif len(commodity) == 2: 
            word = commodity[0] + ' ' + commodity[1]
            if word not in produce:
                produce[word] = []
            produce[word].append(data[1 + 8*j].span.string)

        # 3 words...
        elif len(commodity) == 3:
            word = commodity[0] + ' ' + commodity[1] + ' ' + commodity[2]
            if word not in produce:
                produce[word] = []
            produce[word].append(data[1 + 8*j].span.string)
        
        # 4 words...
        elif len(commodity) == 4:
            word = commodity[0] + ' ' + commodity[1] + ' ' + commodity[2] + ' ' + commodity[3]
            if word not in produce:
                produce[word] = []
            produce[word].append(data[1 + 8*j].span.string)

        # If there's anything longer than 4 words in the commodities column, show me (there's not)
        else:
            print(len(commodity))


        # Now collect the values for each commodity key
        # Everything repeats after 8 iterations, hence the 8*j.
        # data[2 + 8*j] holds the farm price
        for i in data[2 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        # data[3 + 8*j] holds the Atlanta Retail Price
        for i in data[3 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        # Holds the Chicago Retail Price
        for i in data[4 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        # Los Angeles Retail Price
        for i in data[5 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        
        # New York Retail Price
        for i in data[6 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

        
        # Average Spread
        for i in data[7 + 8*j].string.split(' '):
            if (i != '') & (i != '\n'):
                produce[word].append(i)

The above code creates a dictionary that holds the produce names as keys, and a list of all the available data for that produce as its value pair. This is pretty messy. To clean this up, I will be storing the same keys, but for the values I will put all the data into a dataframe with the date as an index.

In [6]:
# Start an empty dictionary
produce_df_dict = dict()

# Create an empty list to store dataframes for concatenation
df_list = []

# iterate through every key in the produce dictionary and create a data frame from its data. Every 7 elements
# in the values for a given key becomes a row in the df. There are 985 total items in each list
for name in list(produce.keys()):
    for i in range(986):
        df_list.append(pd.DataFrame(produce[str(name)][7*i: 7*(i+1)]).T)
    
    produce_df_dict[str(name)] = pd.concat(df_list)
    produce_df_dict[str(name)].columns = ['Date', 'Farm Price', 'Atlanta Retail', 'Chicago Retail', 'LA Retail', 'NYC Retail', 'Avg Spread']
    produce_df_dict[str(name)].index = produce_df_dict[str(name)]['Date']
    produce_df_dict[str(name)].drop(columns =['Date'], inplace=True)
    produce_df_dict[str(name)]['Farm Price'] = pd.to_numeric(produce_df_dict[str(name)]['Farm Price'].str.replace('$', ''))
    produce_df_dict[str(name)]['Atlanta Retail'] = pd.to_numeric(produce_df_dict[str(name)]['Atlanta Retail'].str.replace('$', ''))
    produce_df_dict[str(name)]['Chicago Retail'] = pd.to_numeric(produce_df_dict[str(name)]['Chicago Retail'].str.replace('$', ''))
    produce_df_dict[str(name)]['LA Retail'] = pd.to_numeric(produce_df_dict[str(name)]['LA Retail'].str.replace('$', ''))
    produce_df_dict[str(name)]['NYC Retail'] = pd.to_numeric(produce_df_dict[str(name)]['NYC Retail'].str.replace('$', ''))
    produce_df_dict[str(name)]['Commodity'] = str(name)

KeyboardInterrupt: 

In [ ]:
produce_df_dict.keys()

In [ ]:
produce_df_dict['Strawberries']

Alright. Now to make one data frame that holds it all.

In [ ]:
full_df = []
for name in list(produce_df_dict.keys()):
    full_df.append(produce_df_dict[name])

In [ ]:
grand_df = pd.concat(full_df)

In [ ]:
grand_df.head()

In [ ]:
grand_df.tail()

Cool. Now to save it in a database.

In [ ]:
import sqlite3
conn = sqlite3.connect('agriculture_prices.db')
c = conn.cursor()

In [ ]:
c.execute('''CREATE TABLE agriculture_prices (
                    date BLOB,
                    farm_price REAL,
                    atlanta_retail REAL,
                    chicago_retail REAL,
                    los_angeles_retail REAL,
                    nyc_retail REAL,
                    avg_spread TEXT,
                    commodity TEXT)''')

conn.commit()


In [ ]:
grand_df['Date'] = grand_df.index

In [ ]:
list(grand_df.iloc[0])

In [ ]:
def insert_row(row_list):
    with conn:
        c.execute('''INSERT INTO agriculture_prices VALUES (:farm_price, :atlanta_retail, :chicago_retail, :los_angeles_retail, :nyc_retail, :avg_spread, :commodity, :date)''', 
                 {'farm_price': row_list[0], 'atlanta_retail': row_list[1], 'chicago_retail': row_list[2], 'los_angeles_retail':row_list[3], 'nyc_retail': row_list[4], 'avg_spread': row_list[5], 'commodity': row_list[6], 'date': row_list[7]})
    

Not the most effient way to do this, but it's straightforward.

In [30]:
for row in range(len(grand_df)):
    insert_row(grand_df.iloc[row])

In [32]:
grand_df = c.execute('''SELECT * FROM agriculture_prices''').fetchall()

In [38]:
grand_df[:5]

[(1.16, 2.23, 1.7, 1.99, 2.54, '82.33%', 'Strawberries', '2019-05-19'),
 (0.91, 2.67, 1.89, 2.47, 2.66, '166.21%', 'Strawberries', '2019-05-12'),
 (0.58, 2.42, 1.89, 2.12, 2.91, '302.59%', 'Strawberries', '2019-05-05'),
 (0.58, 2.79, 1.95, 2.38, 2.96, '334.48%', 'Strawberries', '2019-04-28'),
 (0.69, 2.92, 2.04, 2.38, 3.16, '280.43%', 'Strawberries', '2019-04-21')]

In [39]:
grand_df[-5:]

[(0.28, 1.69, 0.0, 1.24, 1.19, '267.86%', 'Nectarines', '2000-07-23'),
 (0.26, 1.61, 0.0, 1.37, 1.19, '300.96%', 'Nectarines', '2000-07-16'),
 (0.3, 1.71, 0.0, 1.42, 1.19, '260.00%', 'Nectarines', '2000-07-09'),
 (0.33, 1.96, 0.0, 0.92, 1.04, '196.97%', 'Nectarines', '2000-07-02'),
 (0.39, 1.22, 0.0, 0.92, 1.19, '113.46%', 'Nectarines', '2000-06-25')]

Looks like it worked and the database has been created.